**1) 잠재 의미 분석(Latent Semantic Analysis, LSA)**

3. 잠재 의미 분석(Latent Semantic Analysis, LSA)

In [ ]:
import pandas as pd  # 데이터프레임 사용을 위해
from math import log # IDF 계산을 위해
import numpy as np

docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '바나나 길고 노란 바나나',
  '저는 과일이 좋아요'
] 

vocab = list(set(w for doc in docs for w in doc.split()))
print('정렬 전 --------------------------------------')
print(vocab)
print()
vocab.sort()
print('정렬 후 --------------------------------------')
print(vocab)
print()

N = len(docs) # 총 문서의 수
print('총 문서 수 ------------------------------------')
print(N)
print()

# tf(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수.
def tf(t, d): 
  return d.count(t)


# df(t) : 특정 단어 t가 등장한 문서의 수.  
def idf(t):
  df = 0
  for doc in docs:
    df += t in doc
  return log(N/(df + 1))

# idf(d, t) : df(t)에 반비례하는 수.
def tfidf(t, d):
  return tf(t,d)*idf(t)


result=[]
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
  result.append([])
  d = docs[i]
  for j in range(len(vocab)):
    t = vocab[j]
    result[-1].append(tf(t,d))

A = pd.DataFrame(result, columns=vocab)
print('DTM ----------------------------------------')    
print(A)
print(np.shape(A))


In [ ]:
# svd - 특이값 분해 (Singular Value Decomposition)
# [Python NumPy] 선형대수 함수 (Linear Algebra) - 출처 https://rfriend.tistory.com/380
# [선형대수] 특이값 분해 (SVD, Singular Value Decomposition) - 출처 https://rfriend.tistory.com/185
# [SVD (SVD와 Latent Factor 모형] - 출처 https://www.fun-coding.org/recommend_basic6.html

# numpy.linalg.svd - https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html

U, s, VT = np.linalg.svd(A, full_matrices=True)
print('U ------------------------------------------') 
print(U)
print()
print('s ------------------------------------------') 
print(s)
print()
print('VT ------------------------------------------') 
print(VT)
print()

print('U (직교행렬) --------------------------------') 
print(U.round(2)) # 소수점 두번째 자리까지 출력
print(np.shape(U))
print()
print('s ------------------------------------------') 
print(s.round(2)) # 소수점 두번째 자리까지 출력
print(np.shape(s))
print()
S = np.zeros((4,9)) # 대각 행렬의 크기인 4 x 9 의 임의의 행렬 생성
S[:4,:4] = np.diag(s) # 특이값을 대각행렬에 삽입
print('S (대각행렬) --------------------------------') 
print(S.round(2)) # 소수점 두번째 자리까지 출력
print(np.shape(S))
print()

print('VT (전치 행렬) -------------------------------') 
print(VT.round(2)) # 소수점 두번째 자리까지 출력
print(np.shape(VT))
print()

np.allclose(A, np.dot(np.dot(U,S), VT).round(2))

In [ ]:
S=S[:2,:2]
print('S (대각행렬) --------------------------------') 
print(S.round(2))
print()

U=U[:,:2]
print('U (직교행렬) --------------------------------')
print(U.round(2))
print()

VT=VT[:2,:]
print('VT (전치 행렬) -------------------------------') 
print(VT.round(2))
print()

A_prime=np.dot(np.dot(U,S), VT)
print(A)
print(A_prime.round(2))

4. 실습을 통한 이해

1) 뉴스그룹 데이터에 대한 이해

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers','footers','quotes'))
documents = dataset.data
print('뉴스그룹 데이터 개수 -------------------------') 
print(len(documents))
print()
print('첫번재 훈련용 데이터 -------------------------') 
print(documents[1])
print()
print('target_names ---------------------------------') 
print(dataset.target_names)
print()

2) 텍스트 전처리

In [ ]:
#  import nltk
#  nltk.download('stopwords')

# 1. 텍스트 전처리
news_df = pd.DataFrame({'document': documents})

# 특수문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]"," ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

print('원문 -----------------------------------------') 
print(news_df['document'][1])
print()
print('전처리 ---------------------------------------') 
print(news_df['clean_doc'][1])
print()

from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK 로 부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
print('불용어 제거 전 -------------------------------') 
print(tokenized_doc[1])
print()

tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words]) # 불용어 제거
print('불용어 제거 후 -------------------------------') 
print(tokenized_doc[1])
print()

3) TF-IDF 행렬 만들기

In [ ]:
# 역토큰화 (토큰화 작업을 역으로 되돌림)
detokenized_doc = []
for i in range(len(news_df)):
  t = ' '.join(tokenized_doc[i])
  detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

print('역토큰화 ---------------------------------------') 
print(news_df['clean_doc'][1])
print()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english',
                             max_features = 1000, # 상위 1000개의 단어를 보존
                             max_df = 0.5,
                             smooth_idf = True)

X = vectorizer.fit_transform(news_df['clean_doc'])
print(X.shape) # TF-IDE 행렬의 크기 확인

4) 토픽 모델링(Topic Modeling)


In [ ]:
from sklearn.decomposition import TruncatedSVD

svd_model = TruncatedSVD(n_components=30, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
print(len(svd_model.components_))
print()

print(np.shape(svd_model.components_))
print()

terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.
print(len(terms))
print(terms[:10])
print()

def get_topics(components, feature_names, n=5):
  for idx, topic in enumerate(components):
    print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(svd_model.components_, terms)

In [41]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object